In [1]:
import pandas as pd
import sweetviz
import pandas_profiling
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn.impute import SimpleImputer
import numpy as np
import datetime
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)

# Reading in the cleaned dataframes

In [3]:
fault = pd.read_csv('..\data\cleaned_fault_data.csv', low_memory=False)

In [4]:
#forgot to drop the index when downloading as a CSV, will drop that now
fault = fault.drop(columns='Unnamed: 0',axis=1)
fault.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Location,time_next_5246,time_until_derate,derate_soon
0,1211418,108604426,2000-03-18 19:14:10,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,96,3,True,126,2015,105427130,36.935,-86.507,2000-03-18 19:14:46,"36.935 , -86.507",NaN,NaN,False
1,1211417,108604425,2000-03-18 19:14:10,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,829,3,True,126,2015,105427130,36.935,-86.507,2000-03-18 19:14:46,"36.935 , -86.507",NaN,NaN,False
2,1211420,108604488,2000-03-18 19:20:47,High Voltage (Left Fuel Level Sensor),NaN,NaN,CECU3B-NAMUX4,PACCR,829,3,False,126,2015,105427130,36.929,-86.496,2000-03-18 19:20:43,"36.929 , -86.496",NaN,NaN,False
3,1211419,108604487,2000-03-18 19:20:47,High Voltage (Fuel Level),NaN,NaN,CECU3B-NAMUX4,PACCR,96,3,False,126,2015,105427130,36.929,-86.496,2000-03-18 19:20:43,"36.929 , -86.496",NaN,NaN,False
4,1211422,108608408,2000-03-19 02:59:58,Not Reporting Data Wheel Sensor ABS Axle 2 Right,AAAI000032*AAAM000038*BB41275 *A82J140721A_9...,5W26153559,EC80ESP,BNDWS,792,7,False,13,1849,105381862,36.758,-86.171,2000-03-19 02:59:53,"36.758 , -86.171",NaN,NaN,False


In [5]:
onboard = pd.read_csv('..\data\cleaned_onboard_data.csv', low_memory=False)

In [6]:
#forgot to drop the index when downloading as a CSV, will drop that now
onboard = onboard.drop(columns='Unnamed: 0',axis=1)
onboard.head()

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0.0,14.21,False,66.48672,423178.7,100.4,11.0,0.0,96.74375,0.0,1632.2,43.2,12300.907429,0.0,NaN,False,78.8,1023,True,NaN,0.0,3276.75,NaN,0.0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


# Join the dataframes

In [7]:
bigg = onboard.merge(fault,how='outer',left_on='FaultId',right_on='RecordID')
bigg.head()

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Location,time_next_5246,time_until_derate,derate_soon
0,1,0.0,14.21,False,66.48672,423178.7,100.4,11.0,0.0,96.74375,0.0,1632.2,43.2,12300.907429,0.0,NaN,False,78.8,1023,True,NaN,0.0,3276.75,NaN,0.0,1.0,990349.0,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,111.0,17.0,True,2.0,1439,105354361.0,38.857,-84.626,2015-02-21 11:34:25,"38.857 , -84.626",NaN,NaN,False
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,2.0,990360.0,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,629.0,12.0,True,127.0,1439,105354361.0,38.857,-84.626,2015-02-21 11:35:10,"38.857 , -84.626",NaN,NaN,False
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,3.0,990364.0,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,1807.0,2.0,False,127.0,1369,105336226.0,41.421,-87.767,2015-02-21 11:35:26,"41.421 , -87.767",NaN,NaN,False
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN,4.0,990370.0,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,1807.0,2.0,True,127.0,1369,105336226.0,41.421,-87.767,2015-02-21 11:36:08,"41.421 , -87.767",NaN,NaN,False
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN,5.0,990416.0,2015-02-21 11:39:41,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,4364.0,17.0,False,2.0,1674,105427130.0,38.416,-89.442,2015-02-21 11:39:37,"38.416 , -89.442",NaN,NaN,False


# Feature Engineering
1. Drop uneccessary columns (explanations listed in previous notebooks)
2. Forward filling any LtD data (using the last known data here seems the best option. will fill with global mode later)
3. Active = True
4. Concatenate spn & fmi columns to get unique codes 
5. Get dummy variables for the above

In [8]:
bigg['EventTimeStamp'] = pd.to_datetime(bigg['EventTimeStamp'])

In [9]:
bigg = bigg.sort_values('EventTimeStamp')

In [10]:
bigg['DistanceLtd'] = bigg.groupby('EquipmentID')['DistanceLtd'].ffill()

bigg['FuelLtd'] = bigg.groupby('EquipmentID')['FuelLtd'].ffill()

bigg['EngineTimeLtd'] = bigg.groupby('EquipmentID')['EngineTimeLtd'].ffill()

### Active = True
From Michael: "Active = true means that it is a fault light being switched on.
Whereas, active = false means that it is being switched off. While there might be some value in the active=false rows, I think that for your first pass, it would make sense just to throw them out."

In [11]:
bigg = bigg[bigg['active']==True]

In [12]:
bigg['spn'] = bigg['spn'].astype(int)

In [13]:
bigg['fmi'] = bigg['fmi'].astype(int)

In [14]:
bigg['spn-fmi'] = bigg['spn'].astype(str) + '-' + bigg['fmi'].astype(str)

In [15]:
dummies = pd.get_dummies(bigg['spn-fmi'])

In [16]:
bigg = pd.concat([bigg,dummies],axis=1)

In [17]:
bigg = bigg.drop(columns=['CruiseControlActive','LampStatus','ParkingBrake','eventDescription',
                          'Latitude','Longitude','LocationTimeStamp','ecuSoftwareVersion','ecuModel',
                          'ecuSerialNumber','MCTNumber','ESS_Id', 'ServiceDistance','CruiseControlSetSpeed',
                          'Location','time_next_5246','activeTransitionCount','Throttle',
                          'RecordID','spn','fmi','spn-fmi'],axis=1)

In [18]:
bigg.head()

,FaultId,AcceleratorPedal,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,TurboBoostPressure,EventTimeStamp,ecuMake,active,EquipmentID,time_until_derate,derate_soon,...,939-4,939-5,94-1,94-17,94-18,94-4,940-3,940-5,941-3,941-4,941-5,942-5,95-15,95-16,96-13,96-3,96-4,96-9,97-15,97-16,97-3,97-4,976-9,98-18,98-5
1154194,1211418,0.0,14.4275,274765.40,183.2,15.0,25.52,190.8500,597.625,5673.10,68.0,37866.421934,0.673641,NaN,True,127.4,0.000000,NaN,0.58,2000-03-18 19:14:10,PACCR,True,2015,NaN,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1154193,1211417,0.0,14.4275,274765.40,183.2,15.0,25.52,190.8500,597.625,5673.10,68.0,37866.421934,0.673641,NaN,True,127.4,0.000000,NaN,0.58,2000-03-18 19:14:10,PACCR,True,2015,NaN,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1154199,1211423,23.2,14.3550,418316.50,170.6,6.0,41.18,173.1875,1279.750,8909.25,92.4,58344.246889,2.113383,32.0,True,82.4,46.641670,NaN,14.50,2000-03-19 03:58:23,BNDWS,True,1849,NaN,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1154208,1211432,48.0,14.7175,55971.24,168.8,37.0,22.62,188.3750,600.000,1096.50,75.6,7238.842569,1.056692,NaN,True,118.4,0.145634,NaN,0.87,2000-03-19 07:32:53,EATON,True,2283,NaN,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1154211,1211435,0.0,14.4275,306348.10,186.8,14.0,19.14,208.0625,599.625,8180.75,63.6,44400.453468,0.528346,NaN,True,116.6,0.000000,NaN,0.00,2000-03-19 08:40:03,PACCR,True,2034,NaN,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Train-Test Split
Needs to be 
1. Grouped by truck
2. Ordered by time, taking the last 20-25% observations for testing is common practice

In [19]:
#I want to make sure I split it into the first 80% of data (sorted by time) and the last 20% of data, this is a one time thing
# commenting out so i don't do this again
def timesplit(df):
    '''splitting my dataframe into train/test ordered by EventTimeStamp'''
    df = df.sort_values('EventTimeStamp')
    nrow = df.shape[0]
    testdf = df.iloc[: round(nrow*0.8),]
    traindf = df.iloc[round(nrow*0.8) : ,]
    return testdf, traindf

In [20]:
# commenting out so i don't do this again
test = pd.DataFrame()
train = pd.DataFrame()

In [21]:
ts = bigg.groupby('EquipmentID').apply(timesplit) #faster than for loop

In [27]:
train = pd.concat([x[0] for x in ts])

In [29]:
test = pd.concat([x[1] for x in ts])

In [23]:
# # commenting out so i don't do this again
# for truck, df in bigg.groupby('EquipmentID'):
#     truck_train, truck_test = timesplit(df)
#     test = test.append(truck_test)
#     train = train.append(truck_train) 

# TRAINING dataset missing values


In [28]:
### How I tested our data
# bigg[['Throttle']].describe()
# bigg[['Throttle']].median()
# bigg['Throttle'].value_counts()
# bigg['Throttle'].isna().sum()

## Creating new class: Group Imputer

In [31]:
class GroupImputer(BaseEstimator, TransformerMixin):
    def __init__(self,strategy='mean'):
        self.strategy = strategy
    def fit(self, df):
        imputer = {}
        if self.strategy == 'mean':
            self.overall_ = df.mean()
        if self.strategy == 'median':
            self.overall_ = df.median()
        if self.strategy == 'mode':
            self.overall_ = df.mode().iloc[0]
        for truck, columns in df.groupby('EquipmentID'):
            if self.strategy == 'mean':
                imputer[truck] = columns.mean()
            if self.strategy == 'median':
                imputer[truck] = columns.median()
            if self.strategy == 'mode':
                imputer[truck] = columns.mode().iloc[0] #iloc is in case there's more than one mode
        self.imputer_ = imputer
        return self
    def transform(self,df):
        new_df = df.copy()
        for truck in self.imputer_:
            val = self.imputer_[truck]
            new_df.loc[new_df.EquipmentID == truck] = new_df.loc[new_df.EquipmentID == truck].fillna(val)
        new_df = new_df.fillna(self.overall_)
        return new_df
            
    

In [30]:
# test= bigg[(bigg['EquipmentID']=='1460') | (bigg['EquipmentID']=='1693')]

In [31]:
# test = test[['BarometricPressure','EngineCoolantTemperature','FuelLevel','EquipmentID']]

In [32]:
# gi = GroupImputer(strategy='mode')

In [33]:
# gi.fit(test)

In [34]:
# gi.transform(test)

In [35]:
# test[test['EquipmentID']=='1460']['FuelLevel'].isna().mean()


### Fill NaN Strategy



##### Impute Mean:
* BarometricPressure
* EngineLoad: a lot of NaNs, high std deviation, most often either 100 or 0 split pretty evenly. is measured engine load compared to theoretical maximum. i'd be willing to run a .describe on everything that's under 100 because if an engine is overloaded that's going to be something that the ECU measures. even when doing this for <90, std dev is high and the mean doesn't change much. I think going with the mean even with the standard deviation is the right call here since even dropping everything above 80 only drops the mean from 29 down to 21
* Speed: people are either going 60-65mph or are stopped completely (which is the most common). do some drive the same routes in the same trucks often? if so, interpolate grouped by truck is probably a good call

##### Impute Median:
* EngineOilPressure
* EngineOilTemperature
* FuelLevel: possibly median
* IntakeManifoldTemperature: I feel comfortable using the median, mean if that doesn't work
* TurboBoostPressure: high std dev but good spread, going with median
* EngineCoolantTemperature: median is 183.2, most frequent is 185
* FuelRate: leaning median or mean, since it's a small range it doesn't _super_ matter if a few mpg are there or not



##### Impute Most Frequent:
* AcceleratorPedal
##### Impute Constant:
* EngineRpm: could possibly do a constant of around 600 but I don't love it
* FuelTemperature: vast majority is 32 so I feel comfortable using this constant
* SwitchedBatteryVoltage: 3276.75 as a constant, is the large majority of the time so seems to be the 'normal'. my assumption is if it were a problem, it would have been measured.
* IgnStatus: replace all NaN with true, since that's what most of them are at.




##### Forward Fill: (do on the og data set before train/test split grouped by truck)
* DistanceLtd: lifetime to date distance that the truck has driven
* EngineTimeLtd: possibly another forward fill (similar to DistanceLtd)
* FuelLtd: (?) unless this is too similar to DistanceLtd or EngineTimeLtd

## Transforming the Training dataset

In [32]:
gi_mean = GroupImputer(strategy='mean')
gi_median = GroupImputer(strategy='median')
gi_mode = GroupImputer(strategy='mode')

In [33]:
mean_col = ['BarometricPressure','EngineLoad','Speed','EquipmentID']
median_col = ['EngineOilPressure','EngineOilTemperature','FuelLevel','IntakeManifoldTemperature','TurboBoostPressure','EngineCoolantTemperature','FuelRate','EquipmentID']
mode_col = ['AcceleratorPedal','EngineRpm','FuelTemperature','SwitchedBatteryVoltage','IgnStatus','DistanceLtd','FuelLtd','EngineTimeLtd','EquipmentID']

In [34]:
ct = ColumnTransformer(transformers=[
    ('mean',gi_mean,mean_col),
    ('median',gi_median,median_col),
    ('mode',gi_mode,mode_col)
], remainder = 'passthrough')

Just a test to make sure the Group Imputer works

In [39]:
# test= bigg[(bigg['EquipmentID']=='1460') | (bigg['EquipmentID']=='1693')]

In [40]:
# ct.fit(test)

In [41]:
# ct.transform(test)

Putting it on the training df we have

In [35]:
ct.fit(train)

C:\Users\zylst\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
C:\Users\zylst\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:993: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)


ColumnTransformer(remainder='passthrough',
                  transformers=[('mean', GroupImputer(),
                                 ['BarometricPressure', 'EngineLoad', 'Speed',
                                  'EquipmentID']),
                                ('median', GroupImputer(strategy='median'),
                                 ['EngineOilPressure', 'EngineOilTemperature',
                                  'FuelLevel', 'IntakeManifoldTemperature',
                                  'TurboBoostPressure',
                                  'EngineCoolantTemperature', 'FuelRate',
                                  'EquipmentID']),
                                ('mode', GroupImputer(strategy='mode'),
                                 ['AcceleratorPedal', 'EngineRpm',
                                  'FuelTemperature', 'SwitchedBatteryVoltage',
                                  'IgnStatus', 'DistanceLtd', 'FuelLtd',
                                  'EngineTimeLtd', 'EquipmentID'])]

In [36]:
ct.transform(train)

array([[14.2825, 29.181818181818183, 15.130145000000002, ..., 0, 0, 0],
       [14.2825, 29.181818181818183, 15.130145000000002, ..., 0, 0, 0],
       [14.2825, 29.181818181818183, 15.130145000000002, ..., 0, 0, 0],
       ...,
       [14.065, 83.0, 65.96243, ..., 0, 0, 0],
       [14.1375, 53.0, 66.83624, ..., 0, 0, 0],
       [14.4275, 12.0, 4.378725, ..., 0, 0, 0]], dtype=object)

## Putting together the training data set

In [53]:
train.head()

,FaultId,AcceleratorPedal,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,TurboBoostPressure,EventTimeStamp,ecuMake,active,EquipmentID,derate_soon,0-0,...,939-4,939-5,94-1,94-17,94-18,94-4,940-3,940-5,941-3,941-4,941-5,942-5,95-15,95-16,96-13,96-3,96-4,96-9,97-15,97-16,97-3,97-4,976-9,98-18,98-5
4967,4968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,2015-02-25 06:17:50,CMMNS,True,1327,False,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4968,4969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,2015-02-25 06:17:50,CMMNS,True,1327,False,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29481,30593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,2015-04-22 09:10:12,CMMNS,True,1327,False,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29482,30594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,2015-04-22 09:10:12,CMMNS,True,1327,False,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29483,30595,0.0,14.065,517683.5,64.4,28.0,41.76,NaN,640.625,NaN,NaN,NaN,1.585037,NaN,True,66.2,NaN,3276.75,0.0,2015-04-22 09:10:14,CMMNS,True,1327,False,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
imputed_columns = mean_col + median_col + mode_col

In [38]:
non_imputed_columns = [x for x in train.columns if x not in imputed_columns]

In [39]:
train = pd.DataFrame(ct.transform(train),columns=(imputed_columns + non_imputed_columns))

In [40]:
test = pd.DataFrame(ct.transform(test),columns=(imputed_columns + non_imputed_columns))

In [41]:
# # commenting out so i don't do this again 
# test.to_csv('../data/testing_data.csv', index=False)

In [42]:
# # commenting out so i don't do this again 
# train.to_csv('../data/training_data.csv', index=False)